## Evaluation Phase

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
#
from evaluation import *

#import environment
sys.path.append(r'../virl')
import virl

In [5]:
#load test environment
env = virl.Epidemic(problem_id=2)

policy_random = policy_generator(env)
states, rewards, actions = exec_agent(policy_random, env)

plot_stats(states, property=[1, 2, 3])

TypeError: list indices must be integers or slices, not list